# Sprint 4 — Webinar 11 (Teórico)  
**Tema:** Journey → Funnel → Cohorts (SQL en PostgreSQL)  
**Versión:** 2025-11-02T15:18:05

> Esta clase teórica usa **PostgreSQL** como motor SQL externo. Incluye scripts de **creación** y **carga mínima** de datos para ejecutar los ejemplos de funneles y cohorts.


<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w1s1_2.png" width="400">
</div>

## Agenda y Objetivos de Aprendizaje

1) **Understand the User Journey and Prepare Data**  
- **LO:** Definir qué es un *user journey* y su importancia.  
- **LO:** Identificar etapas del journey y métricas (conversión, drop-off).  
- **LO:** Reconocer campos relevantes (user_id, timestamp, event_type) y validar calidad de datos.

2) **Building Funnel Analyses with SQL**  
- **LO:** Usar CTEs para estructurar etapas de un funnel.  
- **LO:** Contar usuarios por etapa y calcular *drop-off* y *conversion*.  
- **LO:** Interpretar resultados para priorizar mejoras.

3) **Analyzing Retention with Cohorts**  
- **LO:** Agrupar usuarios por cohort (semanal/mensual).  
- **LO:** Construir tabla de retención (cohort × periodo).  
- **LO:** Segmentar retención por atributos (plan, channel, device) y exportar a Google Sheets para *heatmap*.


## Requisitos previos y conexión a PostgreSQL

- Tener un servidor PostgreSQL accesible (local o remoto).  
- Usuario con permisos de creación en un esquema de práctica.  
- Conexión por `psql`, un cliente gráfico (DBeaver), o extensiones como `ipython-sql` (opcional).
- Puedes usar la página web: https://sqliteonline.com/

# 0) Scripts de creación e inserción de datos (PostgreSQL)

> Ejecuta primero este bloque de **DDL** y **seed data** en tu PostgreSQL.  
Incluye un esquema mínimo para journeys/funnels (e-commerce) y para cohorts.


In [ ]:
# --- PostgreSQL DDL ---
# Copia/ejecuta en tu servidor
print(r'''
-- ==============================================
-- Esquema base para journeys, funnels y cohorts
-- ==============================================

CREATE SCHEMA IF NOT EXISTS s4w11;

-- Usuarios con atributos para segmentación
CREATE TABLE IF NOT EXISTS s4w11.users (
  user_id        BIGSERIAL PRIMARY KEY,
  signup_ts      TIMESTAMP NOT NULL,
  plan           TEXT CHECK (plan IN ('free','paid')) NOT NULL,
  channel        TEXT CHECK (channel IN ('organic','ads','referral','email')) NOT NULL,
  device         TEXT CHECK (device IN ('web','android','ios')) NOT NULL
);

-- Tabla de eventos tipo GA4 minimalista
-- props permite anexar claves específicas por evento (JSONB)
CREATE TABLE IF NOT EXISTS s4w11.events (
  event_id    BIGSERIAL PRIMARY KEY,
  user_id     BIGINT REFERENCES s4w11.users(user_id),
  event_name  TEXT NOT NULL,
  event_ts    TIMESTAMP NOT NULL,
  props       JSONB DEFAULT '{}'::jsonb
);

-- Índices recomendados para analítica de eventos
CREATE INDEX IF NOT EXISTS idx_events_user_ts ON s4w11.events (user_id, event_ts);
CREATE INDEX IF NOT EXISTS idx_events_name_ts ON s4w11.events (event_name, event_ts);
CREATE INDEX IF NOT EXISTS idx_events_gin_props ON s4w11.events USING GIN (props);
''')

In [ ]:
# --- PostgreSQL seed data ---
# Copia/ejecuta en tu servidor
print(r'''
-- ======================================================
-- Seed de usuarios (cohortes de enero y febrero 2021)
-- ======================================================
INSERT INTO s4w11.users (signup_ts, plan, channel, device) VALUES
  ('2021-01-02 09:10','free','organic','web'),
  ('2021-01-03 14:20','paid','ads','ios'),
  ('2021-01-05 08:05','free','referral','android'),
  ('2021-01-10 18:42','free','organic','web'),
  ('2021-01-12 11:11','paid','email','web'),
  ('2021-02-01 09:15','free','ads','android'),
  ('2021-02-05 16:00','paid','referral','ios'),
  ('2021-02-09 10:30','free','organic','web'),
  ('2021-02-12 21:45','free','email','android'),
  ('2021-02-15 07:50','paid','ads','web');

-- Tomamos los IDs generados (1..10) para construir eventos
-- Flujo típico: session_start -> view_item -> add_to_cart -> begin_checkout -> purchase

-- Enero (usuarios 1..5)
INSERT INTO s4w11.events (user_id, event_name, event_ts, props) VALUES
  (1,'session_start','2021-01-02 09:12','{{"utm":"organic"}}'),
  (1,'view_item','2021-01-02 09:14','{{"sku":"A101"}}'),
  (1,'add_to_cart','2021-01-02 09:15','{{"sku":"A101"}}'),
  (1,'begin_checkout','2021-01-02 09:16','{{"cart_value":29.9}}'),
  (1,'purchase','2021-01-02 09:18','{{"order_id":"O-0001","value":29.9}}'),

  (2,'session_start','2021-01-03 14:21','{{"utm":"ads"}}'),
  (2,'view_item','2021-01-03 14:22','{{"sku":"B222"}}'),
  (2,'add_to_cart','2021-01-03 14:23','{{"sku":"B222"}}'),
  (2,'begin_checkout','2021-01-03 14:24','{{"cart_value":58.0}}'),
  (2,'purchase','2021-01-03 14:27','{{"order_id":"O-0002","value":58.0}}'),

  (3,'session_start','2021-01-05 08:06','{{"utm":"referral"}}'),
  (3,'view_item','2021-01-05 08:07','{{"sku":"C333"}}'),
  (3,'add_to_cart','2021-01-05 08:08','{{"sku":"C333"}}'),
  (3,'begin_checkout','2021-01-05 08:09','{{"cart_value":105.0}}'),
  -- (3) abandona antes de purchase

  (4,'session_start','2021-01-10 18:43','{{"utm":"organic"}}'),
  (4,'view_item','2021-01-10 18:46','{{"sku":"D444"}}'),
  -- (4) no agrega a carrito

  (5,'session_start','2021-01-12 11:12','{{"utm":"email"}}'),
  (5,'view_item','2021-01-12 11:13','{{"sku":"A101"}}'),
  (5,'add_to_cart','2021-01-12 11:14','{{"sku":"A101"}}');
  -- (5) no checkout/purchase

-- Febrero (usuarios 6..10)
INSERT INTO s4w11.events (user_id, event_name, event_ts, props) VALUES
  (6,'session_start','2021-02-01 09:17','{{"utm":"ads"}}'),
  (6,'view_item','2021-02-01 09:18','{{"sku":"E555"}}'),
  -- (6) no agrega

  (7,'session_start','2021-02-05 16:01','{{"utm":"referral"}}'),
  (7,'view_item','2021-02-05 16:02','{{"sku":"B222"}}'),
  (7,'add_to_cart','2021-02-05 16:03','{{"sku":"B222"}}'),
  (7,'begin_checkout','2021-02-05 16:04','{{"cart_value":58.0}}'),
  (7,'purchase','2021-02-05 16:05','{{"order_id":"O-0007","value":58.0}}'),

  (8,'session_start','2021-02-09 10:31','{{"utm":"organic"}}'),
  (8,'view_item','2021-02-09 10:32','{{"sku":"C333"}}'),
  (8,'add_to_cart','2021-02-09 10:34','{{"sku":"C333"}}'),
  -- (8) no checkout

  (9,'session_start','2021-02-12 21:46','{{"utm":"email"}}'),
  -- (9) rebote

  (10,'session_start','2021-02-15 07:51','{{"utm":"ads"}}'),
  (10,'view_item','2021-02-15 07:52','{{"sku":"A101"}}'),
  (10,'add_to_cart','2021-02-15 07:53','{{"sku":"A101"}}'),
  (10,'begin_checkout','2021-02-15 07:54','{{"cart_value":29.9}}'),
  (10,'purchase','2021-02-15 07:56','{{"order_id":"O-0010","value":29.9}}');
''')

# 1) Understand the User Journey and Prepare Data

### 1.1 ¿Qué es un *user journey*?
Un *user journey* es la representación de un proceso multi-etapa visto desde la perspectiva del usuario (ej.: descubrimiento → evaluación → conversión → uso/retención).  
Sirve para conectar métricas de producto con impacto de negocio (tasa de conversión, *drop-off*, tiempo entre etapas, LTV).

**Ejercicio (discursivo):** Analiza 3 journeys y define etapas + eventos:
- **E-commerce:** visita → *view_item* → *add_to_cart* → *begin_checkout* → *purchase*  
- **Educación:** *landing* → registro → primera lección → completar módulo → certificación  
- **Servicios públicos:** ingresar a portal → solicitar turno → adjuntar documentos → pago → confirmación

> Boceta cada etapa como un evento con `user_id`, `event_ts`, `event_name` y metadatos (`props`).

### 1.2 Identificar eventos y campos clave en BD
Campos mínimos para reconstruir journeys:  
- `user_id` (agrupar por usuario)  
- `event_ts` (orden temporal)  
- `event_name` (tipo de evento)  
- `props` (JSONB con detalles: sku, valor, utm, etc.)

**Ejemplo: listar tipos de evento únicos**
```sql
SELECT DISTINCT event_name
FROM s4w11.events
ORDER BY 1;
```
**Ejercicio:** Revisa la tabla `s4w11.events` y marca cuáles campos sirven para ordenar eventos, agrupar usuarios y calcular conversiones.


### 1.3 Validación de calidad de datos
Antes de construir consultas:
- **Duplicados:** ¿múltiples eventos idénticos?  
- **Faltantes:** ¿etapas ausentes (p. ej., carrito sin checkout)?  
- **Tiempos:** ¿timestamps fuera de orden o en el futuro?

**Snippets útiles (PostgreSQL):**
```sql
-- 1) Posibles duplicados exactos por usuario/ts/evento
SELECT user_id, event_name, event_ts, COUNT(*) AS n
FROM s4w11.events
GROUP BY user_id, event_name, event_ts
HAVING COUNT(*) > 1
ORDER BY n DESC, event_ts;

-- 2) Eventos "viajeros en el tiempo" (más allá de ahora)
SELECT *
FROM s4w11.events
WHERE event_ts > NOW();

-- 3) Usuarios con carrito pero sin checkout/purchase
WITH cart_users AS (
  SELECT DISTINCT user_id FROM s4w11.events WHERE event_name='add_to_cart'
),
checkout_users AS (
  SELECT DISTINCT user_id FROM s4w11.events WHERE event_name='begin_checkout'
),
purchase_users AS (
  SELECT DISTINCT user_id FROM s4w11.events WHERE event_name='purchase'
)
SELECT cu.user_id,
       (cu.user_id IN (SELECT * FROM checkout_users)) AS has_checkout,
       (cu.user_id IN (SELECT * FROM purchase_users)) AS has_purchase
FROM cart_users cu
ORDER BY 1;
```
Documenta observaciones antes de escribir el funnel.


# 2) Building Funnel Analyses with SQL

### 2.1 Extraer y filtrar eventos relevantes (CTEs)
Las **CTEs** (*Common Table Expressions*, `WITH ... AS (...)`) ayudan a dividir consultas complejas en pasos legibles.


In [ ]:
# --- Funnel con CTEs (conteos por etapa) ---
print(r'''
-- Filtrado por ventana temporal (enero y febrero 2021)
WITH base AS (
  SELECT *
  FROM s4w11.events
  WHERE event_ts >= '2021-01-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
), cte_session AS (
  SELECT DISTINCT user_id FROM base WHERE event_name = 'session_start'
), cte_view AS (
  SELECT DISTINCT user_id FROM base WHERE event_name = 'view_item'
), cte_cart AS (
  SELECT DISTINCT user_id FROM base WHERE event_name = 'add_to_cart'
), cte_checkout AS (
  SELECT DISTINCT user_id FROM base WHERE event_name = 'begin_checkout'
), cte_purchase AS (
  SELECT DISTINCT user_id FROM base WHERE event_name = 'purchase'
)
SELECT
  (SELECT COUNT(*) FROM cte_session)  AS session_users,
  (SELECT COUNT(*) FROM cte_view)     AS view_users,
  (SELECT COUNT(*) FROM cte_cart)     AS cart_users,
  (SELECT COUNT(*) FROM cte_checkout) AS checkout_users,
  (SELECT COUNT(*) FROM cte_purchase) AS purchase_users;
''')

### 2.2 Reescribir funnel con “una CTE por etapa” (ejercicio)
**Ejercicio:** Reescribe un funnel (signup → add_to_cart → purchase) usando una CTE por etapa.  
**Pista:** usa `SELECT DISTINCT user_id` en cada CTE y al final cuenta usuarios por etapa.


In [ ]:
# --- Funnel con drop-off y conversión ---
print(r'''
-- Tabla de funnel con drop-off y tasas de conversión
WITH base AS (
  SELECT *
  FROM s4w11.events
  WHERE event_ts >= '2021-01-01'::timestamp
    AND event_ts <  '2021-03-01'::timestamp
), s AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='session_start'
), v AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='view_item'
), c AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='add_to_cart'
), ch AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='begin_checkout'
), p AS (
  SELECT DISTINCT user_id FROM base WHERE event_name='purchase'
), counts AS (
  SELECT
    (SELECT COUNT(*) FROM s)  AS n_session,
    (SELECT COUNT(*) FROM v)  AS n_view,
    (SELECT COUNT(*) FROM c)  AS n_cart,
    (SELECT COUNT(*) FROM ch) AS n_checkout,
    (SELECT COUNT(*) FROM p)  AS n_purchase
)
SELECT
  n_session,
  n_view,
  n_cart,
  n_checkout,
  n_purchase,
  (n_session - n_view)     AS drop_session_to_view,
  (n_view - n_cart)        AS drop_view_to_cart,
  (n_cart - n_checkout)    AS drop_cart_to_checkout,
  (n_checkout - n_purchase)AS drop_checkout_to_purchase,
  ROUND(100.0 * n_purchase / NULLIF(n_session,0), 2) AS cr_session_to_purchase_pct
FROM counts;
''')

### 2.3 Interpretación de resultados
- Identifica el **cuello de botella** (mayor caída).  
- Propón hipótesis (p. ej., fricción en checkout móvil, valor del carrito, medios de pago).  
- Plantea experimento A/B o UX research para esa etapa.


# 3) Analyzing Retention with Cohorts

Definimos **cohort** por fecha de *signup* (p. ej., mes o semana) y medimos si los usuarios regresan en periodos posteriores (retención D+7, W+1, M+1).  
Conectamos con el funnel calculando métricas por cohort (ej., conversión por cohort mensual).

In [ ]:
# --- Retención semanal por cohort (tabla larga) ---
print(r'''
-- Asignar cohort mensual por fecha de signup del usuario
-- y calcular primeras interacciones posteriores por semana
WITH u AS (
  SELECT
    user_id,
    date_trunc('month', signup_ts)::date AS cohort_month
  FROM s4w11.users
),
events_norm AS (
  SELECT e.user_id, e.event_ts::date AS event_date
  FROM s4w11.events e
),
grid AS (
  -- Consideramos 0..8 semanas desde el signup para ejemplo
  SELECT generate_series(0, 8) AS week_offset
),
retention AS (
  SELECT
    u.cohort_month,
    g.week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM events_norm en
        WHERE en.user_id = u.user_id
          AND en.event_date >= (u.cohort_month + (g.week_offset || ' week')::interval)::date
          AND en.event_date <  (u.cohort_month + ((g.week_offset+1) || ' week')::interval)::date
      )
    ) AS retained_users,
    COUNT(DISTINCT u.user_id) AS cohort_users
  FROM u
  CROSS JOIN grid g
  GROUP BY u.cohort_month, g.week_offset
)
SELECT
  cohort_month,
  week_offset,
  retained_users,
  cohort_users,
  ROUND(100.0 * retained_users / NULLIF(cohort_users,0), 2) AS retention_pct
FROM retention
ORDER BY cohort_month, week_offset;
''')

In [ ]:
# --- Retención semanal por cohort (tabla ancha para heatmap en Sheets) ---
print(r'''
-- Pivote de retención por cohort (filas) × week_offset (columnas)
-- Nota: para un pivote "nativo" se puede usar crosstab (extensión tablefunc).
-- Aquí mostramos una alternativa simple con agregación condicional.
WITH base AS (
  WITH u AS (
    SELECT user_id, date_trunc('month', signup_ts)::date AS cohort_month
    FROM s4w11.users
  ),
  grid AS (SELECT generate_series(0, 8) AS week_offset),
  retention AS (
    SELECT
      u.cohort_month,
      g.week_offset,
      COUNT(DISTINCT u.user_id) FILTER (
        WHERE EXISTS (
          SELECT 1 FROM s4w11.events e
          WHERE e.user_id = u.user_id
            AND e.event_ts::date >= (u.cohort_month + (g.week_offset || ' week')::interval)::date
            AND e.event_ts::date <  (u.cohort_month + ((g.week_offset+1) || ' week')::interval)::date
        )
      )::float AS retained_users,
      COUNT(DISTINCT u.user_id)::float AS cohort_users
    FROM u
    CROSS JOIN grid g
    GROUP BY u.cohort_month, g.week_offset
  )
  SELECT cohort_month, week_offset,
         CASE WHEN cohort_users = 0 THEN 0
              ELSE ROUND(100.0 * retained_users / cohort_users, 2) END AS retention_pct
  FROM retention
)
SELECT
  cohort_month,
  MAX(CASE WHEN week_offset=0 THEN retention_pct END) AS w0,
  MAX(CASE WHEN week_offset=1 THEN retention_pct END) AS w1,
  MAX(CASE WHEN week_offset=2 THEN retention_pct END) AS w2,
  MAX(CASE WHEN week_offset=3 THEN retention_pct END) AS w3,
  MAX(CASE WHEN week_offset=4 THEN retention_pct END) AS w4,
  MAX(CASE WHEN week_offset=5 THEN retention_pct END) AS w5,
  MAX(CASE WHEN week_offset=6 THEN retention_pct END) AS w6,
  MAX(CASE WHEN week_offset=7 THEN retention_pct END) AS w7,
  MAX(CASE WHEN week_offset=8 THEN retention_pct END) AS w8
FROM base
GROUP BY cohort_month
ORDER BY cohort_month;
''')

In [ ]:
# --- Retención segmentada por plan (free vs paid) ---
print(r'''
-- Retención segmentada por plan (free vs paid)
WITH u AS (
  SELECT user_id,
         date_trunc('month', signup_ts)::date AS cohort_month,
         plan
  FROM s4w11.users
),
grid AS (SELECT generate_series(0, 8) AS week_offset),
retention AS (
  SELECT
    u.cohort_month,
    u.plan,
    g.week_offset,
    COUNT(DISTINCT u.user_id) FILTER (
      WHERE EXISTS (
        SELECT 1 FROM s4w11.events e
        WHERE e.user_id = u.user_id
          AND e.event_ts::date >= (u.cohort_month + (g.week_offset || ' week')::interval)::date
          AND e.event_ts::date <  (u.cohort_month + ((g.week_offset+1) || ' week')::interval)::date
      )
    )::float AS retained_users,
    COUNT(DISTINCT u.user_id)::float AS cohort_users
  FROM u
  CROSS JOIN grid g
  GROUP BY u.cohort_month, u.plan, g.week_offset
)
SELECT
  cohort_month,
  plan,
  week_offset,
  CASE WHEN cohort_users = 0 THEN 0
       ELSE ROUND(100.0 * retained_users / cohort_users, 2) END AS retention_pct
FROM retention
ORDER BY cohort_month, plan, week_offset;
''')

### 3.1 → 3.3 Exportar y *Heatmap* en Google Sheets
1) Exporta la tabla de retención (ancha) a CSV desde PostgreSQL:  
```sql
-- Ajusta ruta en servidor
COPY (
  /* Usa aquí la consulta 'retention (tabla ancha)' */
  WITH u AS (
    SELECT user_id, date_trunc('month', signup_ts)::date AS cohort_month
    FROM s4w11.users
  ),
  grid AS (SELECT generate_series(0, 8) AS week_offset),
  retention AS (
    SELECT
      u.cohort_month,
      g.week_offset,
      COUNT(DISTINCT u.user_id) FILTER (
        WHERE EXISTS (
          SELECT 1 FROM s4w11.events e
          WHERE e.user_id = u.user_id
            AND e.event_ts::date >= (u.cohort_month + (g.week_offset || ' week')::interval)::date
            AND e.event_ts::date <  (u.cohort_month + ((g.week_offset+1) || ' week')::interval)::date
        )
      )::float AS retained_users,
      COUNT(DISTINCT u.user_id)::float AS cohort_users
    FROM u CROSS JOIN grid g
    GROUP BY u.cohort_month, g.week_offset
  )
  SELECT
    cohort_month,
    MAX(CASE WHEN week_offset=0 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w0,
    MAX(CASE WHEN week_offset=1 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w1,
    MAX(CASE WHEN week_offset=2 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w2,
    MAX(CASE WHEN week_offset=3 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w3,
    MAX(CASE WHEN week_offset=4 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w4,
    MAX(CASE WHEN week_offset=5 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w5,
    MAX(CASE WHEN week_offset=6 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w6,
    MAX(CASE WHEN week_offset=7 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w7,
    MAX(CASE WHEN week_offset=8 THEN ROUND(100.0 * retained_users / NULLIF(cohort_users,0),2) END) AS w8
  FROM retention
  GROUP BY cohort_month
  ORDER BY cohort_month
) TO '/tmp/retention_weekly.csv' WITH CSV HEADER;
```
2) Sube el CSV a Google Sheets.  
3) Selecciona la matriz (W0..W8) → **Formato condicional → Escala de color** para un *heatmap* claro.


# 4) Ejercicios

**4.1 (Journey):** Para educación y servicios públicos, escribe un listado de eventos por etapa con `user_id`, `event_ts`, `event_name` y `props` sugeridos.  
**4.2 (Funnel):** Construye un funnel `session_start → view_item → add_to_cart → purchase` y calcula *drop-off* por etapa en febrero 2021.  
**4.3 (Cohorts):** Genera cohorts **semanales** por `MIN(signup_ts)` y produce retención en `week_offset = 0..6`.  
**4.4 (Segmentación):** Compara retención `free` vs `paid` y escribe una hipótesis de negocio.


## Cierre
**Kahoot de repaso (5 min)**
- Guarda tus consultas clave en un script comentado.
- Exporta una tabla final y redacta una recomendación ejecutiva.
- **Tarea:** rehacer los ejercicios usando tus propias tablas reales si existen.

## Siguientes Pasos
- **Próxima sesión:** Iniciaremos con la práctica.
- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados, se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.


---
**Licencia:** CC BY 4.0 — Uso educativo.  
**Créditos:** Preparado para Sprint 4 — Webinar 11 (Teórico).
